In [71]:
%pylab inline
import scipy as S
from scipy.special import legendre
from scipy.integrate import quad
from scipy import interpolate as I
import astropy as A
from astropy import convolution as conv
#from scipy.stats import norm

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['var']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [72]:
fg=0.7
f1=1
f2=0
sig_v=1.5
sig8=1.55

In [73]:
#Charge les fichier de la prediction CLPT
fXi=loadtxt("../CLPT/data/Xi_r_simu.dat")
fV12=loadtxt("../CLPT/data/V12_simu.dat")
fSigma=loadtxt("../CLPT/data/Sigma_simu.dat")

xi_x=fXi[:,0]
xi_y=fXi[:,1]+f1*fXi[:,2]+f2*fXi[:,3]+f1*f1*fXi[:,4]+f1*f2*fXi[:,5]+f2*f2*fXi[:,6] #F0,F',F'',F'²,F'_F'',F''²
v12_x=fV12[:,0]
v12_y=fV12[:,1]+f1*fV12[:,2]+f2*fV12[:,3]+f1*f1*fV12[:,4]+f1*f2*fV12[:,5]
sig_x=fSigma[:,0]
sig_par=fSigma[:,1]+f1*fSigma[:,2]+f2*fSigma[:,3]+f1*f1*fSigma[:,4]
sig_per=fSigma[:,5]+f1*fSigma[:,6]+f2*fSigma[:,7]+f1*f1*fSigma[:,8]



In [74]:
V12 = I.interp1d(v12_x, v12_y, kind='cubic',bounds_error=False, fill_value=0)
Sig_par = I.interp1d(sig_x, sig_par, kind='cubic',bounds_error=False, fill_value=0)
Sig_per = I.interp1d(sig_x, sig_per, kind='cubic',bounds_error=False, fill_value=0)
Xi_r = I.interp1d(xi_x, xi_y, kind='cubic',bounds_error=False, fill_value=0)

In [75]:
#Fonction 
def gauss (x,moy,var2) :
    return 1./(sqrt(2*pi*var2))*exp(-(x-moy)**2/(2*var2))

def Sigma (mu, r):
    if (sig_x[0]<=r<=sig_x[-1]):
        return (mu**2*(Sig_par(r))+(1.-mu*mu)*Sig_per(r))/(1.+Xi_r(r))+sig_v**2
    else :
        return 0

def fXis (y, spar, rp):
    r = sqrt(rp*rp+y*y)
    if (sig_x[0]<=r<=sig_x[-1]):
        v = fg*V12(r)/(1.+Xi_r(r))
        mu_r = y/r
        x = spar-y
        moy = mu_r*v
        var = fg**2*Sigma(mu_r,r)
        if (var>0):
            return (1.+Xi_r(r))*gauss(x, moy, var)
        else :
            return 0
    else :
        return 0
    
def Xis (spar, rp):
    return quad(fXis,-150,150,args=(spar,rp),epsabs=0)[0]-1
   # return S.integrate.quadrature(fXis,-150,150,args=(spar,rp))

#Fonction multipole
def fmultipole (mu, s, l):
    rp = s*sqrt(1-mu*mu)
    spi = s*mu
    Xi_s= Xis(rp,spi)
    return Xi_s*legendre(l)(mu)

def multipole (s, l):
    return quad(fmultipole,-1,1,args=(s,l),epsabs=0)[0]*(2*l+1)/2
    #return S.integrate.quadrature(fmultipole,-1,1,args=(s,l))[0]
#quad(func, a, b, args, full_output, epsabs, epsrel, limit, points, weight, wvar, wopts, maxp1, limlst)

In [76]:
multipole(30,0)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:38: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.


0.14940622950074042

In [14]:
i=0
x = array([])
r=array([])
mu_r=array([])
for spar in range(1,130,26):
    for y in range (-150,150):
        x = append(x,spar-y)
        r = append(r,sqrt(spar*spar+y*y))
        mu_r = append(mu_r,y/r[i])
        i+=1

for i in range(1,100):
    if (sig_x[0]<=r[i]<=sig_x[-1]):
        var = fg**2*Sigma(mu_r[i],r[i])    
        xi=conv.convolve_fft(x,conv.Gaussian1DKernel(stddev=var))

In [15]:
def gaus(bins, mu=0, sigma=1):
        return 1.0/(np.sqrt(sigma * 2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma))
    
def moy(rpar,r):
    if (sig_x[0]<=r<=sig_x[-1]):
        return rpar/r*fg*V12(r)/(1.+Xi_r(r))
    else : 
        return 0

def var(rpar,r):                   #sigma2
    S = fg**2*Sigma(rpar/r,r)
    if S > 0 : 
        return S
    else : 
        return 0
    
def convolve_var(rpar, r, xi, moy, var, t=1): #rpar=vector, xi=1+xi(r)(vector)
    n = rpar.size
    g = np.zeros(n)
    s = np.zeros(n)
    rolled = xi
    xx = np.roll(rpar,int(n/2))

    for i in range(n):
        g = gaus(rpar, moy(xx[i],r[i]),var(xx[i],r[i]))
        s[i] = sum(rolled * g)
        rolled = np.roll(rolled,t)

    s = np.roll(s,int(n/2))
               
    return s

In [70]:
#save pour plot
arr = empty((99,3))
out = empty((99*99,3))
rpv = arange(1,100)
rpar = arange(1,100)
for rp in rpv:
    r = np.sqrt(rp**2+rpar**2)
    opx = 1 + Xi_r(r)
    result = convolve_var(rpar,r,opx,moy,var)
    arr = np.array([rpar*0+rp,rpar,result]).T
    out[(rp-1)*99:rp*99,:] = arr
savetxt("outmodel.dat",out)

In [68]:
out

array([[ 1.        ,  1.        ,  0.48456875],
       [ 1.        ,  2.        ,  0.48415507],
       [ 1.        ,  3.        ,  0.483894  ],
       ...,
       [99.        , 97.        ,  0.46145674],
       [99.        , 98.        ,  0.46155667],
       [99.        , 99.        ,  0.47798753]])

In [206]:
def kernel (rp):
    k=np.zeros(130)
    rpp=np.zeros(130)
    i=0
    for rpar in range(1,131):
        rpp[i]=rpar
        r=sqrt(rpar*rpar+rp*rp)
        if (sig_x[0]<=r<=sig_x[-1]):
            mu = rpar/r
            moy=mu*fg*V12(r)/(1.+Xi_r(r))
            var2=fg**2*Sigma(mu,r)
            xi[i] = 1+Xi_r(r)
            if var2>0:
                k[i] = gauss(rpar,moy,var2)
            else: 
                k[i]=0
        else :
            k[i]=0
            xi[i]=0
        i+=1
    return xi,k,rpp